# Imports

In [1]:
_cache = './cache'
_data = './data'

## News Data

In [2]:
from src.news import NewsDataManager

In [3]:
newsData = NewsDataManager( _data, _cache )

src1 = '/news/Combined_News_DJIA.csv'
src2 = '/news/github.csv'

if ( newsData.save_exists('news') ):
    newsData.load('news')
else:
    newsData.process_data(src1 )
    newsData.save('news')

Read Saved Data                                                                                                     


In [4]:
newsData.data.head()

,Date,Headline
0,2008-08-08,"[eorgia, down, two, russian, warplan, countri,..."
1,2008-08-08,"[reak, musharraf, impeach]"
2,2008-08-08,"[ussia, today, column, troop, roll, south, oss..."
3,2008-08-08,"[ussian, tank, move, toward, capit, south, oss..."
4,2008-08-08,"[fghan, children, rape, impun, UN, offici, say..."


## Embeddings

In [5]:
import spacy
import gensim 
from gensim.models import Word2Vec 
import numpy as np
import gensim.downloader as api

In [8]:
#model_glove_twitter = api.load("glove-twitter-25")
#encoding_size = model_glove_twitter['bob'].shape[0]
encoding_size = 25
encoding_size

25

In [9]:
create_custom = False

if create_custom:
    if ( newsData.save_exists('news+embeddings') ):
        newsData.load('news+embeddings')
    else:
        # Create a word to vec encoder 
        sentences = newsData.data.Headline.values
        news_model = Word2Vec(sentences, size=300, min_count=1)
        # Map the embeddings onto it
        newsData.use_embedding_map( news_model.wv, encoding_size )
        # Save output
        newsData.save('news+embeddings')
else:
    if ( newsData.save_exists('news+glove') ):
        newsData.load('news+glove')
    else:
        newsData.use_embedding_map( model_glove_twitter, encoding_size )
        newsData.save('news+glove')

Read Saved Data                                                                                                     


In [10]:
newsData.data.head()

,Date,Headline,Embedding
0,2008-08-08,"[georgia, down, two, russian, warplan, countri...","[0.08687834850408965, 0.14082443879710305, -0...."
1,2008-08-08,"[break, musharraf, impeach]","[0.10069001, 0.61090666, -0.494177, -0.2415999..."
2,2008-08-08,"[russia, today, column, troop, roll, south, os...","[-0.16874400824308394, 0.15566962528973818, -0..."
3,2008-08-08,"[russian, tank, move, toward, capit, south, os...","[-0.2749368376814975, -0.06807107535692361, -0..."
4,2008-08-08,"[afghan, children, rape, impun, UN, offici, sa...","[-0.19116646194687256, 0.36368216344943416, 0...."


## Stock Data

In [11]:
from src.stocks import StockDataManager

In [12]:
stockData = StockDataManager( 'SPY', _data, _cache )
stockData.data.head()

Stock Data Manager
Read Saved Data                                                                                                     


,Date,Open,High,Low,Close,Adj Close,Volume,forward_gains,backward_gains
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,26.299288,1003200,0.023610,NaN
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,26.486324,480500,0.015983,NaN
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,26.542448,201300,0.013898,NaN
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,26.822998,529400,-0.003498,NaN
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,26.935240,531500,-0.006289,0.024181


## Merge Data

In [13]:
combinedData = newsData.data.set_index('Date').join( stockData.data.set_index('Date') )
combinedData.head()

,Headline,Embedding,Open,High,Low,Close,Adj Close,Volume,forward_gains,backward_gains
Date,,,,,,,,,,
2008-08-08,"[georgia, down, two, russian, warplan, countri...","[0.08687834850408965, 0.14082443879710305, -0....",126.580002,129.929993,126.379997,129.369995,101.503098,260811700,0.001312,0.035043
2008-08-08,"[break, musharraf, impeach]","[0.10069001, 0.61090666, -0.494177, -0.2415999...",126.580002,129.929993,126.379997,129.369995,101.503098,260811700,0.001312,0.035043
2008-08-08,"[russia, today, column, troop, roll, south, os...","[-0.16874400824308394, 0.15566962528973818, -0...",126.580002,129.929993,126.379997,129.369995,101.503098,260811700,0.001312,0.035043
2008-08-08,"[russian, tank, move, toward, capit, south, os...","[-0.2749368376814975, -0.06807107535692361, -0...",126.580002,129.929993,126.379997,129.369995,101.503098,260811700,0.001312,0.035043
2008-08-08,"[afghan, children, rape, impun, UN, offici, sa...","[-0.19116646194687256, 0.36368216344943416, 0....",126.580002,129.929993,126.379997,129.369995,101.503098,260811700,0.001312,0.035043


In [14]:
combinedData = combinedData.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'] )
combinedData.head()

,Headline,Embedding,forward_gains,backward_gains
Date,,,,
2008-08-08,"[georgia, down, two, russian, warplan, countri...","[0.08687834850408965, 0.14082443879710305, -0....",0.001312,0.035043
2008-08-08,"[break, musharraf, impeach]","[0.10069001, 0.61090666, -0.494177, -0.2415999...",0.001312,0.035043
2008-08-08,"[russia, today, column, troop, roll, south, os...","[-0.16874400824308394, 0.15566962528973818, -0...",0.001312,0.035043
2008-08-08,"[russian, tank, move, toward, capit, south, os...","[-0.2749368376814975, -0.06807107535692361, -0...",0.001312,0.035043
2008-08-08,"[afghan, children, rape, impun, UN, offici, sa...","[-0.19116646194687256, 0.36368216344943416, 0....",0.001312,0.035043


In [15]:
# Create normalized data for regression
regressionData = combinedData.copy( deep = True )
def normalize( data, col ):
    data[col] = ( data[col] - data[col].min() ) / ( data[col].max() - data[col].min() )
normalize( regressionData, 'forward_gains' )
normalize( regressionData, 'backward_gains' )
regressionData.head()

,Headline,Embedding,forward_gains,backward_gains
Date,,,,
2008-08-08,"[georgia, down, two, russian, warplan, countri...","[0.08687834850408965, 0.14082443879710305, -0....",0.591182,0.596434
2008-08-08,"[break, musharraf, impeach]","[0.10069001, 0.61090666, -0.494177, -0.2415999...",0.591182,0.596434
2008-08-08,"[russia, today, column, troop, roll, south, os...","[-0.16874400824308394, 0.15566962528973818, -0...",0.591182,0.596434
2008-08-08,"[russian, tank, move, toward, capit, south, os...","[-0.2749368376814975, -0.06807107535692361, -0...",0.591182,0.596434
2008-08-08,"[afghan, children, rape, impun, UN, offici, sa...","[-0.19116646194687256, 0.36368216344943416, 0....",0.591182,0.596434


In [16]:
# Create binary dataset aswell
binaryData = combinedData.copy( deep = True )
binaryData['forward_gains'] = binaryData['forward_gains'] > 0
binaryData['backward_gains'] = binaryData['backward_gains'] > 0
binaryData.head()

,Headline,Embedding,forward_gains,backward_gains
Date,,,,
2008-08-08,"[georgia, down, two, russian, warplan, countri...","[0.08687834850408965, 0.14082443879710305, -0....",True,True
2008-08-08,"[break, musharraf, impeach]","[0.10069001, 0.61090666, -0.494177, -0.2415999...",True,True
2008-08-08,"[russia, today, column, troop, roll, south, os...","[-0.16874400824308394, 0.15566962528973818, -0...",True,True
2008-08-08,"[russian, tank, move, toward, capit, south, os...","[-0.2749368376814975, -0.06807107535692361, -0...",True,True
2008-08-08,"[afghan, children, rape, impun, UN, offici, sa...","[-0.19116646194687256, 0.36368216344943416, 0....",True,True


## Create Tensorflow Model

In [21]:
import tensorflow as tf

In [22]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(encoding_size,)),
    tf.keras.layers.Dense(20, activation=tf.nn.relu),
    tf.keras.layers.Dense(20, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.softmax)
])
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

## Seperate Dataset

In [23]:
import random

In [24]:
# Training on forward gains for now
use_column = 'backward_gains'

In [25]:
positive_examples = binaryData['Embedding'][binaryData[use_column]].dropna().values 
positive_examples = np.stack( positive_examples )
positive_examples.shape

(28418, 25)

In [26]:
negative_examples = np.array( binaryData['Embedding'][~binaryData[use_column]].dropna().values )
negative_examples = np.stack( negative_examples )
negative_examples.shape

(21300, 25)

In [27]:
X = np.concatenate([positive_examples, negative_examples])
y = np.concatenate([np.zeros(positive_examples.shape[0]) + 1, np.zeros(negative_examples.shape[0]) ])
print(X.shape, y.shape)

(49718, 25) (49718,)


## Training

In [28]:
model.fit( x=X, y=y, validation_split=0.1, epochs = 4)

Train on 44746 samples, validate on 4972 samples
Epoch 1/4
44746/44746 [==============================] - 4s 99us/sample - loss: 5.5645 - accuracy: 0.6351 - val_loss: 15.2492 - val_accuracy: 0.0000e+00
Epoch 2/4
44746/44746 [==============================] - 4s 80us/sample - loss: 5.5645 - accuracy: 0.6351 - val_loss: 15.2492 - val_accuracy: 0.0000e+00
Epoch 3/4
44746/44746 [==============================] - 4s 83us/sample - loss: 5.5645 - accuracy: 0.6351 - val_loss: 15.2492 - val_accuracy: 0.0000e+00
Epoch 4/4
44746/44746 [==============================] - 4s 84us/sample - loss: 5.5645 - accuracy: 0.6351 - val_loss: 15.2492 - val_accuracy: 0.0000e+00
